# **КОНТРОЛЬНАЯ РАБОТА**
## Склифасовский Денис Олегович	33 вариант
### Пояснения
В файлах по вариантам (csdataNN.csv, где NN соответствует вашему номеру варианта) находятся исходные данные. **В нечетных вариантах находятся данные для кредитного скоринга**, в четных – данные для оценки стоимости кристаллов фианита. Вам необходимо построить не менее 3 моделей и определить, какая из них лучшая.
Контрольную работу можно выполнять в программе Deductor, а также, по желанию студента, в программе Loginom Academic или на языках R или Python.

Модели должны быть неодинаковыми. Нужно применить разные инструменты (например, в Deductor варианты – линейная регрессия, логистическая регрессия, дерево решений, нейронная сеть). Допускается, чтобы две из трех моделей были построены на одном инструменте, но их параметры должны отличаться (например, две нейронных сети с разным числом слоев и нейронов).

Сравните результаты обучения и выберите по этим результатам лучшую (для ваших данных) модель.

### Описание данных

Выходным полем является для задачи скоринга поле SeriousDlqin2yrs (Были ли в последние два года задержки платежа 2 года и более). Его значение – 1 (были -> плохой клиент) или 0 (не было -> хороший клиент). Остальные поля цифровые, входные (кроме поля RefNum, если оно есть – это условный номер). Их описание, если вам интересно, находится в конце этого документа в приложении.

### Оформление результатов
Вам нужно сдать два файла: собственно файл с моделями и файл MS Word с пояснениями. 

Файлы с исходными данными изменять нельзя, их вы не сдаете, при проверке я использую архивные копии.

1. **Файл с моделями** - результаты представляются в файле скрипта («*.R» или «*.py»), который содержит все этапы построения от загрузки данных до вывода результатов; в тексте модели следует разделить строкой-примечанием, например, «# модель 1»

2. **Файл с текстом пояснений (*.doc, *.docx, *.odf)** - в файле нужно привести результаты обучения моделей (матрицы ошибок, важнейшие показатели) и затем сделать выбор лучшей модели и обосновать, почему она лучшая. Приветствуется расчет различных показателей качества классификации, а также сравнение с константной моделью.

## ***1. Загрузка данных***

In [1]:
import pandas as pd

df = pd.read_csv('csdata33.csv', delimiter=';', encoding='cp1251')

## ***2. Предварительный анализ данных***

### ***2.1. Обзор структуры данных***

In [2]:
df.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30.59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60.89DaysPastDueNotWorse,NumberOfDependents
0,0,"0,65818014",38,1,"0,085113375",3042,2,1,0,0,0
1,0,"0,392994586",58,2,"0,436102527",5500,15,0,1,0,0
2,0,"0,881836362",50,3,"0,567858035",4000,9,0,1,0,1
3,0,"0,9999999",75,1,"0,134078212",3400,1,0,1,0,1
4,0,"0,469056842",31,0,"0,048211004",9250,4,0,0,0,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 11 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   SeriousDlqin2yrs                      5000 non-null   int64 
 1   RevolvingUtilizationOfUnsecuredLines  5000 non-null   object
 2   age                                   5000 non-null   int64 
 3   NumberOfTime30.59DaysPastDueNotWorse  5000 non-null   int64 
 4   DebtRatio                             5000 non-null   object
 5   MonthlyIncome                         5000 non-null   int64 
 6   NumberOfOpenCreditLinesAndLoans       5000 non-null   int64 
 7   NumberOfTimes90DaysLate               5000 non-null   int64 
 8   NumberRealEstateLoansOrLines          5000 non-null   int64 
 9   NumberOfTime60.89DaysPastDueNotWorse  5000 non-null   int64 
 10  NumberOfDependents                    5000 non-null   int64 
dtypes: int64(9), object(2)
memory 

### ***2.2. Проверка на отсутствующие значения***

In [4]:
df.isnull().sum()

SeriousDlqin2yrs                        0
RevolvingUtilizationOfUnsecuredLines    0
age                                     0
NumberOfTime30.59DaysPastDueNotWorse    0
DebtRatio                               0
MonthlyIncome                           0
NumberOfOpenCreditLinesAndLoans         0
NumberOfTimes90DaysLate                 0
NumberRealEstateLoansOrLines            0
NumberOfTime60.89DaysPastDueNotWorse    0
NumberOfDependents                      0
dtype: int64

## ***3. Подготовка данных***

In [5]:
for column in df.select_dtypes(include='object').columns:
    df[column] = df[column].str.replace(',', '.').astype(float)

df.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30.59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60.89DaysPastDueNotWorse,NumberOfDependents
0,0,0.658180,38,1,0.085113,3042,2,1,0,0,0
1,0,0.392995,58,2,0.436103,5500,15,0,1,0,0
2,0,0.881836,50,3,0.567858,4000,9,0,1,0,1
3,0,1.000000,75,1,0.134078,3400,1,0,1,0,1
4,0,0.469057,31,0,0.048211,9250,4,0,0,0,1


In [6]:
from sklearn.model_selection import train_test_split

# Разделение на признаки и целевую переменную
features = df.drop("SeriousDlqin2yrs", axis=1)
target = df["SeriousDlqin2yrs"]

# Разделение на обучающий и тестовый наборы
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)


70% данных будет использовано для обучения модели, а оставшиеся 30% данных будут использованы для тестирования модели. 

In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

## ***4. Построение и обучение моделей***

In [44]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

### ***4.1. Нейронная сеть***

In [48]:
model_nn = keras.Sequential([
    layers.Dense(256, activation='LeakyReLU', input_dim=x_train_scaled.shape[1]),
    layers.Dropout(0.2),
    layers.Dense(128, activation='LeakyReLU'),
    layers.Dropout(0.2),
    layers.Dense(64, activation='LeakyReLU'),
    layers.Dropout(0.2),
    layers.Dense(128, activation='LeakyReLU'),
    layers.Dropout(0.2),
    layers.Dense(64, activation='LeakyReLU'),
    layers.Dropout(0.2),
    layers.Dense(1, activation='sigmoid'),
])

model_nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_nn.fit(x_train_scaled, y_train, epochs=20, batch_size=32, validation_data=(x_test_scaled, y_test))

Epoch 1/20
110/110 [==============================] - 3s 19ms/step - loss: 0.2176 - accuracy: 0.9354 - val_loss: 0.1943 - val_accuracy: 0.9400
Epoch 2/20
110/110 [==============================] - 1s 11ms/step - loss: 0.1786 - accuracy: 0.9406 - val_loss: 0.2016 - val_accuracy: 0.9413
Epoch 3/20
110/110 [==============================] - 1s 10ms/step - loss: 0.1743 - accuracy: 0.9420 - val_loss: 0.1898 - val_accuracy: 0.9400
Epoch 4/20
110/110 [==============================] - 1s 10ms/step - loss: 0.1691 - accuracy: 0.9411 - val_loss: 0.1872 - val_accuracy: 0.9400
Epoch 5/20
110/110 [==============================] - 1s 10ms/step - loss: 0.1668 - accuracy: 0.9434 - val_loss: 0.1880 - val_accuracy: 0.9407
Epoch 6/20
110/110 [==============================] - 1s 10ms/step - loss: 0.1670 - accuracy: 0.9431 - val_loss: 0.1905 - val_accuracy: 0.9400
Epoch 7/20
110/110 [==============================] - 1s 11ms/step - loss: 0.1648 - accuracy: 0.9437 - val_loss: 0.1863 - val_accuracy: 0.9420

### ***4.2. Дерево решений***

In [54]:
model_tree = DecisionTreeClassifier(criterion='gini', max_depth=5, min_samples_split=2, min_samples_leaf=1, max_features=None, random_state=42)
model_tree.fit(x_train_scaled, y_train)

DecisionTreeClassifier(max_depth=5, random_state=42)

### ***4.3. Логистическая регрессия***

In [56]:
model_lr = LogisticRegression(penalty='l2', C=1.0, solver='liblinear', max_iter=100, random_state=42)
model_lr.fit(x_train_scaled, y_train)

LogisticRegression(random_state=42, solver='liblinear')

## ***5. Оценка производительности***

In [9]:
from sklearn.metrics import classification_report

### ***5.1. Оценка нейронной сети***

In [49]:
y_pred_nn = model_nn.predict(x_test_scaled)
y_pred_nn = (y_pred_nn > 0.5).astype(int)
classification_report_nn1 = classification_report(y_test, y_pred_nn)

print("Метрики ошибки для нейросети (Model_nn):")
print(classification_report_nn1)

47/47 [==============================] - 0s 3ms/step
Метрики ошибки для нейросети (Model_nn):
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      1410
           1       0.58      0.08      0.14        90

    accuracy                           0.94      1500
   macro avg       0.76      0.54      0.55      1500
weighted avg       0.92      0.94      0.92      1500



### ***5.2. Оценка дерева решений***

In [55]:
y_pred_tree = model_tree.predict(x_test_scaled)
classification_report_tree = classification_report(y_test, y_pred_tree)

print("Метрики ошибки для дерева решений:")
print(classification_report_tree)

Метрики ошибки для дерева решений:
              precision    recall  f1-score   support

           0       0.95      0.98      0.96      1410
           1       0.35      0.17      0.23        90

    accuracy                           0.93      1500
   macro avg       0.65      0.57      0.59      1500
weighted avg       0.91      0.93      0.92      1500



### ***5.3. Оценка логистической регрессии***

In [57]:
y_pred_lr = model_lr.predict(x_test_scaled)
classification_report_lr = classification_report(y_test, y_pred_lr)

print("Метрики ошибки для логистической регрессии:")
print(classification_report_lr)

Метрики ошибки для логистической регрессии:
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      1410
           1       0.52      0.14      0.23        90

    accuracy                           0.94      1500
   macro avg       0.73      0.57      0.60      1500
weighted avg       0.92      0.94      0.92      1500



## ***6. Сравнение результатов***

In [61]:
target_names = ['0', '1']

report_nn = classification_report(y_test, y_pred_nn, output_dict=True)
report_tree = classification_report(y_test, y_pred_tree, target_names=target_names, output_dict=True)
report_lr = classification_report(y_test, y_pred_lr, target_names=target_names, output_dict=True)


data1 = {'Модель': ['Model_nn', 'Дерево решений', 'Логистическая регрессия'],
        'Точность': [report_nn['0']['precision'], report_tree['0']['precision'], report_lr['0']['precision']],
        'Полнота': [report_nn['0']['recall'], report_tree['0']['recall'], report_lr['0']['recall']],
        'F1-мера': [report_nn['0']['f1-score'], report_tree['0']['f1-score'], report_lr['0']['f1-score']]}
df1 = pd.DataFrame(data1)

df1

,Модель,Точность,Полнота,F1-мера
0,Model_nn,0.944220,0.996454,0.969634
1,Дерево решений,0.948524,0.980142,0.964074
2,Логистическая регрессия,0.947797,0.991489,0.969151


In [62]:
target_names = ['0', '1']

report_nn = classification_report(y_test, y_pred_nn, output_dict=True)
report_tree = classification_report(y_test, y_pred_tree, target_names=target_names, output_dict=True)
report_lr = classification_report(y_test, y_pred_lr, target_names=target_names, output_dict=True)

data0 = {'Модель': ['Model_nn', 'Дерево решений', 'Логистическая регрессия'],
        'Точность': [report_nn['1']['precision'], report_tree['1']['precision'], report_lr['1']['precision']],
        'Полнота': [report_nn['1']['recall'], report_tree['1']['recall'], report_lr['1']['recall']],
        'F1-мера': [report_nn['1']['f1-score'], report_tree['1']['f1-score'], report_lr['1']['f1-score']]}
df0 = pd.DataFrame(data0)

df0

,Модель,Точность,Полнота,F1-мера
0,Model_nn,0.583333,0.077778,0.137255
1,Дерево решений,0.348837,0.166667,0.225564
2,Логистическая регрессия,0.520000,0.144444,0.226087


## 7. ***Выводы***

1. Нейронная сеть показывает высокую точность (accuracy) на уровне 94%, однако метрика f1-score для класса 1 (плохие клиенты) достаточно низкая - всего 0.14. Это означает, что модель имеет слабую способность предсказывать положительный класс, и доля ложно-положительных и ложно-отрицательных предсказаний велика.

2. Дерево решений показывает немного более низкую точность (accuracy) на уровне 93%, а также низкую метрику f1-score для класса 1 (плохие клиенты) - всего 0.23. Это указывает на недостаточную способность модели предсказывать положительный класс.

3. Логистическая регрессия имеет точность (accuracy) на уровне 94%, а метрика f1-score для класса 1 (плохие клиенты) составляет 0.23. Хотя эта модель также имеет низкую способность предсказывать положительный класс, она показывает немного лучшие результаты, чем две предыдущие модели.

Логистическая регрессия демонстрирует наилучшую производительность среди трех моделей для данной задачи скоринга. 

Но все модели имеют низкую способность предсказывать положительный класс.